# Sciop to Source
* Download dataset from Sciop via Bittorrent
* Upload dataset to Sourc.coop
* Visualize the dataset from the bucket
*

Here we are using a notebook for documentation and visualization, but the real work is done by two CLI commands: (1) download the data using the ARIA2 CLI and (2) push to bucket using the AWS CLI. 

This could be run on any computer:
```
$ export AWS_REQUEST_CHECKSUM_CALCULATION=WHEN_REQUIRED
$ conda install aria2 awscli 
$ aria2c --seed-time=0 https://sciop.net/torrents/9840fa77d032a3821180f7f1726df02431e37e53/estuarine-bathymetric-digital-elevation-models.torrent
$ aws s3 sync s3 estuarine-bathymetry-digital-elevation-models/ s3://pangeo/esip2025
```

## 1. Download the dataset
Obtain from Sciop using bittorrent 

In [ ]:
! aria2c --seed-time=0 https://sciop.net/torrents/9840fa77d032a3821180f7f1726df02431e37e53/estuarine-bathymetric-digital-elevation-models.torrent

## 2. Upload the dataset
Push to Source.coop AWS bucket (in us-west-2 region)

In [ ]:
%%time
! aws s3 sync estuarine-bathymetric-digital-elevation-models s3://pangeo/esip2025 --endpoint-url=https://data.source.coop

## 3. Visualize the dataset
Use the Holoviz tools to interactively explore the data directly from the bucket

In [ ]:
import fsspec
import xarray as xr
import hvplot.xarray

In [ ]:
fs_read = fsspec.filesystem('s3', anon=True, endpoint_url='https://data.source.coop', use_listings_cache=False, skip_instance_cache=True)

In [ ]:
ds = xr.open_dataset(fs_read.open('pangeo/esip2025/Buzzards_Bay/buzzards_bay_m010_30m.nc'))

In [ ]:
ds['Band1'].hvplot(x='x', y='y', crs='EPSG:26919', rasterize=True, tiles='OSM', cmap='viridis')

In [ ]:
fs_read.du('s3://pangeo/esip2025/')/1e9   # GB

In [ ]:
fs_read.ls('s3://pangeo/esip2025/Buzzards_Bay')

In [ ]:
ds

In [ ]:
### Cleanup

In [ ]:
# ! aws s3 rm  s3://pangeo/esip2025 --endpoint-url=https://data.source.coop --recursive
# ! aws s3 ls  s3://pangeo/esip2025 --endpoint-url=https://data.source.coop -